In [1]:
import numpy as np
import pandas as pd
import os
os.chdir("/content/drive/MyDrive/projects/Store_Sales-Time_Series_Forecasting/")

LightGBMの予測をアンサンブルする

In [2]:
newcsv = "2210100708_Ridge_log.csv"

df_ridge = pd.read_csv("result/" + newcsv)
df_lgbm = pd.read_csv("result/2209190750_LightGBM.csv")
df_ridge["model"] = ["Ridge"] * len(df_ridge)
df_lgbm["model"] = ["LightGBM"] * len(df_lgbm)

In [3]:
df_ridge.head()

,store_nbr,family,params,val_RMSLE,test_RMSLE,model
0,1,AUTOMOTIVE,"{'alpha': 10, 'standardscaler_features': ['dco...",0.621155,0.545968,Ridge
1,1,BABY CARE,"{'alpha': 0.01, 'standardscaler_features': ['d...",0.000000,0.000000,Ridge
2,1,BEAUTY,"{'alpha': 10, 'standardscaler_features': ['dco...",0.466292,0.558789,Ridge
3,1,BEVERAGES,"{'alpha': 1.0, 'standardscaler_features': ['dc...",0.250427,0.244350,Ridge
4,1,BOOKS,"{'alpha': 0.01, 'standardscaler_features': ['d...",0.448476,0.178970,Ridge


In [4]:
print(len(df_ridge))
print(len(df_ridge[df_ridge["val_RMSLE"] <= df_lgbm["val_RMSLE"]]))
print(len(df_lgbm[df_ridge["val_RMSLE"] > df_lgbm["val_RMSLE"]]))

1782
1250
532


In [5]:
df_rsub = pd.read_csv("submission/"+newcsv)
df_lsub = pd.read_csv("submission/2209190750_LightGBM.csv")

In [6]:
df_rsub.head()

,id,sales
0,3000888,3.0
1,3000889,0.0
2,3000890,4.0
3,3000891,2248.0
4,3000892,0.0


In [7]:
df_lsub.head()

,id,sales
0,3000888,3.341821
1,3000889,0.000000
2,3000890,4.041330
3,3000891,2215.863840
4,3000892,0.118917


In [8]:
rsub = df_rsub["sales"].tolist()
lsub =np.rint(df_lsub["sales"].tolist())
answer = (rsub + lsub) / 2
df_ans = pd.DataFrame({"id":df_rsub["id"], "sales":np.rint(answer)})
df_ans.head()

,id,sales
0,3000888,3.0
1,3000889,0.0
2,3000890,4.0
3,3000891,2232.0
4,3000892,0.0


schoolの一部をProphetの予測で上書き

In [10]:
df_prophet = pd.read_csv("submission/prophet_school.csv", parse_dates=["date"])
df_prophet.head()

,date,nbr,family,y_pred
0,2017-08-16,9,SCHOOL AND OFFICE SUPPLIES,28.316672
1,2017-08-17,9,SCHOOL AND OFFICE SUPPLIES,27.593552
2,2017-08-18,9,SCHOOL AND OFFICE SUPPLIES,27.242484
3,2017-08-19,9,SCHOOL AND OFFICE SUPPLIES,30.840410
4,2017-08-20,9,SCHOOL AND OFFICE SUPPLIES,34.385314


In [11]:
df_test = pd.read_csv("store-sales-time-series-forecasting/test.csv", index_col=1, parse_dates=True)
dates = df_test.index.tolist()
familys = df_test["family"].tolist()
store_nbrs = df_test["store_nbr"].tolist()
ids = df_test["id"].tolist()
id_dic = dict()

for date, nbr, family, id in zip(dates, store_nbrs, familys, ids):
  if (date, nbr, family) not in id_dic:
    id_dic[(date, nbr, family)] = id

In [12]:
prophet_id = []

p_dates = df_prophet["date"].tolist()
p_familys = df_prophet["family"].tolist()
p_store_nbrs = df_prophet["nbr"].tolist()

for date, nbr, family in zip(p_dates, p_store_nbrs, p_familys):
  id = id_dic[(date, nbr, family)]
  prophet_id.append(id)

In [13]:
ids = df_ans.id.tolist()
old_sales = df_ans.sales.tolist()
new_result = []
for id, sale in zip(ids, old_sales):
  if id not in prophet_id:
    new_result.append([id, sale])

for id, sale in zip(prophet_id, np.rint(df_prophet["y_pred"].tolist())):
  new_result.append([id, sale])

new_result = pd.DataFrame(new_result, columns=["id", "sales"]).sort_values(["id"])
new_result.head()

,id,sales
0,3000888,3.0
1,3000889,0.0
2,3000890,4.0
3,3000891,2232.0
4,3000892,0.0


In [14]:
from datetime import datetime

model_name = "Ridge_log+LightGBM+prophet"
filename = datetime.now().strftime(format="%y%m%d%H%M") + '_' + model_name
new_result.to_csv(f"submission/{filename}.csv", index=None)

In [ ]:
from datetime import datetime
filename = datetime.now().strftime(format="%y%m%d%H%M") + '_' + "Ridge+LightGBM"
df_ans.to_csv(f"submission/{filename}.csv", index=None)